<a href="https://colab.research.google.com/github/oebyakarah/PDF-QA-Agent/blob/main/PDF_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Question-Answering Agent
This is An AI tool that lets you upload a PDF and ask questions about it using Retrieval-Augmented Generation .


In [1]:
!pip install langchain langchain_community
!pip install pypdf faiss-cpu
!pip install transformers sentence-transformers
!pip install huggingface_hub

In [2]:
from google.colab import files

uploaded = files.upload()

Saving AI_Whitepaper_Portfolio.pdf to AI_Whitepaper_Portfolio (2).pdf


In [3]:
!ls /content

'AI_Whitepaper_Portfolio (1).pdf'   AI_Whitepaper_Portfolio.pdf
'AI_Whitepaper_Portfolio (2).pdf'   sample_data


In [4]:
!pip install requests==2.32.4 --force-reinstall

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (151 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Succe

In [1]:
!pip show langchain langchain_community pypdf faiss-cpu transformers sentence-transformers requests

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
---
Name: langchain-community
Version: 0.3.29
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: pypdf
Version: 6.1.0
Summary: A pure-python PDF library capable of splitting, merging, cropping, and transforming PDF files
Home-page: 
Author: 
Author-email: Mathieu Fenniak <biziqe@mathieu.fenniak.net>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: 
---
Nam

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "/content/AI_Whitepaper_Portfolio.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"PDF loaded! Total pages: {len(documents)}")

PDF loaded! Total pages: 2


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 6


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-2643194333.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
!pip install -q langchain langchain_community transformers faiss-cpu pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

query = "Summarize the main points from chapter 1."
result = qa_chain.run(query)
print(result)

Device set to use cpu
/tmp/ipython-input-2324180085.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)
/tmp/ipython-input-2324180085.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)


Embedding generation using Hugging Face Agentic AI and Retrieval-Augmented Generation (RAG): A Practical Whitepaper Prepared by: [Your Name Here] Date: September 2025 This document provides an overview of Agentic AI systems and their integration with Retrieval-Augmented Generation (RAG). It covers the core concepts, methodologies, and a step-by-step case study. This whitepaper is designed to demonstrate skills in Natural Language Processing (NLP), LangChain pipelines, and modern LLM techniques for real-world applications. This whitepaper is designed to demonstrate skills in Natural Language Processing (NLP), LangChain pipelines, and modern LLM techniques for real-world applications. Abstract Results The system successfully answered domain-specific queries with a 92% accuracy rate in a controlled test environment. Compared to a baseline non-RAG LLM, our system showed a 40% improvement in factual correctness and significantly reduced hallucinations. Future Work Future improvements will f

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [8]:
query = "Summarize the Methodolgy section."
result = qa_chain.run(query)
print(result)

Retrieval-Augmented Generation (RAG) addresses this limitation by incorporating external knowledge dynamically during inference.


In [11]:
import textwrap

def wrap_output(text, width=100):
    return "\n".join(textwrap.wrap(str(text), width=width))

questions = [
    "Summarize the Methodology section.",
    "What are the key findings?",
    "Explain the case study in simple terms."
]

for question in questions:
    raw_answer = qa_chain.run(question)
    wrapped_answer = wrap_output(raw_answer)
    print("Question:", question)
    print("Answer:\n", wrapped_answer)
    print("\n" + "="*80 + "\n")

Question: Summarize the Methodology section.
Answer:
 Retrieval-Augmented Generation (RAG): A Practical Whitepaper Prepared by: [Your Name Here] Date:
September 2025 This paper provides an overview of Agentic AI systems and their integration with
Retrieval-Augmented Generation (RAG). It covers the core concepts, methodologies, and a step-by-step
case study. This whitepaper is designed to demonstrate skills in Natural Language Processing (NLP),
LangChain pipelines, and modern LLM techniques for real-world applications.


Question: What are the key findings?
Answer:
 The system successfully answered domain-specific queries with a 92% accuracy rate in a controlled
test environment. Compared to a baseline non-RAG LLM, our system showed a 40% improvement in factual
correctness and significantly reduced hallucinations. Future Work Future improvements will focus on:
- Scaling to multi-modal inputs (text, images, audio). - Implementing advanced reasoning through
ReAct and tool-chaining. - Depl